In [ ]:
#default_exp rescore.feature_extractor

In [ ]:
import pandas as pd

from alphadeep.model.CCS import AlphaCCSModel
from alphadeep.model.RT import AlphaRTModel
from alphadeep.model.msms import pDeepModel, evaluate_msms
from alphadeep.mass_spec.match import PepSpecMatch
from alphabase.peptide.fragment import get_charged_frag_types

class ScoreFeatureExtractor(object):
    def __init__(self,
        msms_model:pDeepModel,
        rt_model:AlphaRTModel,
        ccs_model: AlphaCCSModel,
        ppm=True, tol=20,
        use_GPU = True,
    ):
        self.msms_model = msms_model
        self.rt_model = rt_model
        self.ccs_model = ccs_model
        self.ppm = ppm
        self.tol = tol
        self.GPU = use_GPU

        self.score_feature_list = [
            'COS', 'SA', 'SPC',
            'delta_RT', 'delta_CCS',
        ]

    def _fine_tune_models(self, 
        psm_df:pd.DataFrame, 
        matched_inten_df:pd.DataFrame
    ):
        pass

    def extract_features(self,
        psm_df: pd.DataFrame,
        ms2_file_dict, #raw_name: ms2_file_path or ms_reader object
        ms2_type:str = 'alphapept', #or 'mgf', or 'thermo'
        frag_types_to_match:list = get_charged_frag_types(['b','y'], 2)
    ):
        self.match = PepSpecMatch(psm_df, 
            charged_frag_types=frag_types_to_match
        )

        self.match.match_ms2_centroid(
            ms2_file_dict=ms2_file_dict,
            ms2_type=ms2_type, 
            ppm=True, tol=20,
        )

        self.match.psm_df = self.rt_model.predict(
            self.match.psm_df
        )

        self.match.psm_df = self.ccs_model.predict(
            self.match.psm_df
        )

        predict_inten_df = self.msms_model.predict(
            self.match.psm_df
        )

        self.match.psm_df, eval_metrics_df = evaluate_msms(
            self.match.psm_df, predict_inten_df, 
            self.match.matched_inten_df, 
            charged_frag_types=frag_types_to_match,
            metrics=['COS','SA','SPC'],
            GPU=self.GPU,
        )



            